In [2]:
import sys

from scripts.pfa import Path

sys.path.append('../')

In [3]:
from tqdm.notebook import tqdm
from scripts import centroids_graph_builder, graph_osm_loader, utils, clustering
from heapq import heappop, heappush
from itertools import count
from scripts import pipeline, pfa
from scripts.path_findings import dijkstra_pfa
import networkx as nx

In [4]:
def dijkstra_pfa_min_dst(graph: nx.Graph,
                 start: set[int],
                 ) -> \
        tuple[float, list[int]]:
    adjacency = graph._adj
    c = count()
    push = heappush
    pop = heappop
    dist = {}
    fringe = []
    for s in start:
        dist[s] = 0.0
        push(fringe, (0.0, next(c), s))
    while fringe:
        (d, _, v) = pop(fringe)
        for u, e in adjacency[v].items():
            vu_dist = d + e['length']
            if u not in dist or dist[u] > vu_dist:
                dist[u] = vu_dist
                push(fringe, (vu_dist, next(c), u))
    return dist


Примеры как резолвить комьюнити

In [5]:
GRAPH_ID = 'R13470549'  # R13470549 R2555133 R3766483

In [6]:
# примеры id есть в graph_osm_loader.py
g = graph_osm_loader.get_graph(GRAPH_ID)  # загрузка графа
# N = 1000
# p = 0.01
# g = get_rand_graph(N,p)
len(g.nodes), len(g.edges)

(723, 1046)

In [7]:
utils.get_opt_cluster_count(len(g.nodes))  # оптимальное количество кластеров

128

In [8]:
g.remove_edges_from(nx.selfloop_edges(g))

In [9]:
name = 'cluster'

In [10]:
cms2 = clustering.resolve_k_means_communities(g, resolution=400, max_iteration=100, cluster_name=name, print_log=True)
print(len(cms2))

100%|██████████| 100/100 [00:00<00:00, 3266.92it/s]

554


In [11]:
cls2n = centroids_graph_builder.get_cls2n(g, name=name)  # мапа кластер к соседним кластерам 

In [12]:
g1, cls2c = centroids_graph_builder.build_center_graph(g, cms2, cls2n, log=True, name=name)

find centroids:   0%|          | 0/554 [00:00<?, ?it/s]

find edges:   0%|          | 0/554 [00:00<?, ?it/s]

In [13]:
points = utils.generate_points(g, 1000)

In [14]:
d_clusters = {}
d_nodes = {}
for u in tqdm(g1.nodes()):
    dst = dijkstra_pfa_min_dst(g, {cls2c[u]})
    for v in g1.nodes():
        d_clusters[u, v] = dst[cls2c[v]]
        d_clusters[v, u] = dst[cls2c[v]]

for u, d in tqdm(g.nodes(data=True)):
    c = cls2c[d[name]]
    l = nx.single_source_dijkstra(g, u, c, weight='length')[0]
    d_nodes[u] = l

  0%|          | 0/554 [00:00<?, ?it/s]

  0%|          | 0/723 [00:00<?, ?it/s]

In [15]:
nodes = g.nodes()

def func(u, v):
    c1 = nodes[u]['cluster']
    c2 = nodes[v]['cluster']   
    d1 = d_nodes[u] - d_nodes[v] - d_clusters[c1,c2]
    d2 = abs(d_clusters[c1,c2] - d_nodes[v]) - d_nodes[u] 
    d3 = abs(d_clusters[c1,c2] - d_nodes[u]) - d_nodes[v]
    d4 = d_nodes[v] - d_nodes[u] - d_clusters[c1,c2]
    return max(d1,d2,d3,d4)

In [16]:
def dijkstra_pfa_cls(graph: nx.Graph,
                     start: int,
                     end: int
                     ) -> tuple[float, list[int]]:
    if start == end:
        return 0, [start]
    push = heappush
    pop = heappop
    dist = {start: (0, None)}
    fringe = []
    c = count()
    adjacency = graph._adj
    push(fringe, (0, next(c), 0, func(start, end), start))

    while fringe:
        (_, _, d1, l1, v1) = pop(fringe) 
        if v1 == end:
            break
        for u, e in adjacency[v1].items():
            vu_dist = d1 + e['length']
            if u not in dist or dist[u][0] > vu_dist:
                dist[u] = (vu_dist, v1)
                lu = func(u, end)
                push(fringe, (vu_dist + lu, next(c), vu_dist, lu, u))

    path = [end]
    e = end
    while dist[e][1] is not None:
        e = dist[e][1]
        path = [e] + path
    l = 0
    e = g.edges()
    for i in range(len(path) - 1):
        p1, p2 = path[i], path[i + 1]
        l += e[p1, p2]['length']
    return l, path

In [17]:
class AltPfa(pfa.PathFinding):
    
    def find_path(self, start: int, end: int)-> Path:
        return dijkstra_pfa_cls(self.g, start, end)

stat = pipeline.PfaComparator(
    baseline=dijkstra_pfa.Dijkstra(g),
    test_algorithm=AltPfa(g),
    points=points
).compare()
stat

start 0


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

start 1


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

start 2


find paths:   0%|          | 0/250 [00:00<?, ?it/s]

start 3


find paths:   0%|          | 0/250 [00:00<?, ?it/s]


        err_max:    0.0000
        err_min:    0.0000
        err_median: 0.0000
        err_mean:   0.0000
        
        acceleration_max:    12.6139
        acceleration_min:    0.2152
        acceleration_median: 2.2369
        acceleration_mean:   2.3359
        